# Geodata Manipulation

EDA and data manipulation of GIS data using H3, Geopandas, and Shapely.

In [9]:
import json
from typing import List

import h3
import pandas as pd
import dask.dataframe as dd

In [12]:
def get_geodata(filepath: str):
    gdf = read_file(filepath)
    return (gdf
            .astype({"INTPTLAT": float, "INTPTLON": float, "GEOID": "category"})
            .drop(["STATEFP", "COUNTYFP", "TRACTCE", "ALAND", "AWATER", "NAME", 
                   "NAMELSAD", "MTFCC", "FUNCSTAT"], axis=1)
            .rename({"INTPTLAT": "lat", "INTPTLON": "lon", "GEOID": "geoid"}, axis=1)
           )


def prepare_districts(gdf_districts):
    """Loads a geojson files of polygon geometries and features,
    swaps the latitude and longitude andstores geojson"""    
    return (gdf_districts
            .assign(geom_swap_geojson = lambda x: x["geometry"].map(lambda polygon: transform(
                       lambda x, y: (y, x), polygon)).apply(lambda y: mapping(y))))


def hex_fill_tract(geom_geojson: dict, res: int = 13, flag_swap: bool = False) -> set:
    """Fill a tract with small, res 13 hexagons.

    :param geom_geojson: The polygon to fill.
    :param res: The resolution to fill the polygons with.
    :param flag_swap: A flag indicating whether the polygon is geojson conformant or swapped.
    """
    try:
        set_hexagons = h3.compact(h3.polyfill(geom_geojson, res, geo_json_conformant = flag_swap))
    except ValueError:
        print(f"Error on data of type {geom_geojson['type']}. Continuing.")
        return set()
    return list(set_hexagons)


def hex_fill_df(gdf):
    """Fill the tracts with hexagons."""
    return gdf.assign(hex_fill = gdf["geom_swap_geojson"].apply(hex_fill_tract))

datadir = "../data/zipfiles"
zipfile = os.listdir(datadir)[0]
path = os.path.join(datadir, zipfile)

gdf = (get_geodata(path)
       .pipe(prepare_districts)
       .pipe(hex_fill_df)
      )

KeyboardInterrupt: 

In [2]:
all_tracts = []

for filename in os.listdir("../data/tract_polygons"):
    gdf = read_file(f"../data/tract_polygons/{filename}/{filename}.shp")
    # Unify the CT boundries
    union_poly = unary_union(gdf.geometry)
    
    # Convert to hexagon
    temp  = mapping(g)
    temp['coordinates']=[[[j[1],j[0]] for j in i] for i in temp['coordinates']]  
    gdf['hexes'] = h3.polyfill(temp, APERTURE_SIZE)
    all_tracts.append(gdf)
    
gdf = pd.concat(all_tracts)

In [9]:
APERTURE_SIZE = 3

gdf = read_file(f"../data/tract_polygons/tl_2020_01_tract/tl_2020_01_tract.shp")
union_poly = unary_union(gdf.geometry)
temp  = mapping(union_poly)
temp['coordinates']=[[[j[1],j[0]] for j in i] for i in temp['coordinates']]
hexes = h3.polyfill(temp, APERTURE_SIZE)

In [21]:
import os

import h3
import numpy as np
import pandas as pd
from shapely.ops import transform
from shapely.geometry import mapping
from dask.distributed import Client, LocalCluster
from dask_geopandas import read_parquet

from logging import getLogger

DATA_DIR = "../data"
ZIP_DIR = os.path.join(DATA_DIR, "zipfiles")
PARQUET_DIR = os.path.join(DATA_DIR, "tract_parquet")
TILED_CENSUS_DIR = os.path.join(DATA_DIR, "tiled_states")

LOGGER = getLogger(__name__)


def prepare_districts(gdf):
    """Loads a geojson files of polygon geometries and features,
    swaps the latitude and longitude and stores geojson"""
    return gdf.assign(
        geom_swap_geojson=lambda x: x["geometry"]
        .map(lambda polygon: transform(lambda x, y: (y, x), polygon))
        .apply(lambda y: mapping(y))
    )


def hex_fill_tract(geom_geojson: dict, res: int = 13, flag_swap: bool = False) -> set:
    """Fill a tract with small, res 13 hexagons.

    :param geom_geojson: The polygon to fill.
    :param res: The resolution to fill the polygons with.
    :param flag_swap: A flag indicating whether the polygon is geojson conformant or swapped.
    """
    try:
        set_hexagons = h3.compact(
            h3.polyfill(geom_geojson, res, geo_json_conformant=flag_swap)
        )
    except ValueError:
        LOGGER.debug("Error on data of type %s. Continuing.", geom_geojson["type"])
        return set()
    return list(set_hexagons)


def hex_fill_df(gdf):
    """Fill the tracts with hexagons."""
    return gdf.assign(hex_fill=gdf["geom_swap_geojson"].apply(hex_fill_tract)).drop(
        ["geometry", "geom_swap_geojson"], axis=1
    )


def tile_partition(df: pd.DataFrame):
    """Tile a single tract."""
    return df.pipe(prepare_districts).pipe(hex_fill_df)


cluster = LocalCluster(n_workers=8)
# Get the list of files to read
infiles = set(os.listdir(PARQUET_DIR))
donefiles = set(os.listdir(TILED_CENSUS_DIR))
files_todo = infiles.difference(donefiles)
LOGGER.info("%s states to tile.", len(files_todo))

2022-11-18 07:50:36,456 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://127.0.0.1:39509' processes=2 threads=4, memory=7.77 GiB>>
Traceback (most recent call last):
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/compatibility.py", line 161, in _run
    val = self.callback()
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/client.py", line 1442, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/batched.py", line 156, in send
    raise CommClosedError(f"Comm {self.comm!r} already closed.")
distributed.comm.core.CommClosedError: Comm <TCP (closed) Client->Scheduler local=tcp://127.0.0.1:57804 remote=tcp://127.0.0.1:39509> already closed.
2022-11-18 07:50:41,455 - tornado.application - ERROR - Exception in callback 

In [26]:
client = Client("tcp://127.0.0.1:57804")
client.close()

2022-11-18 07:56:11,454 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://127.0.0.1:39509' processes=2 threads=4, memory=7.77 GiB>>
Traceback (most recent call last):
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/compatibility.py", line 161, in _run
    val = self.callback()
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/client.py", line 1442, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/batched.py", line 156, in send
    raise CommClosedError(f"Comm {self.comm!r} already closed.")
distributed.comm.core.CommClosedError: Comm <TCP (closed) Client->Scheduler local=tcp://127.0.0.1:57804 remote=tcp://127.0.0.1:39509> already closed.
2022-11-18 07:56:16,454 - tornado.application - ERROR - Exception in callback 

OSError: Timed out trying to connect to tcp://127.0.0.1:57804 after 30 s

2022-11-18 07:56:41,454 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://127.0.0.1:39509' processes=2 threads=4, memory=7.77 GiB>>
Traceback (most recent call last):
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/compatibility.py", line 161, in _run
    val = self.callback()
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/client.py", line 1442, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/batched.py", line 156, in send
    raise CommClosedError(f"Comm {self.comm!r} already closed.")
distributed.comm.core.CommClosedError: Comm <TCP (closed) Client->Scheduler local=tcp://127.0.0.1:57804 remote=tcp://127.0.0.1:39509> already closed.
2022-11-18 07:56:46,456 - tornado.application - ERROR - Exception in callback 

2022-11-18 07:57:26,456 - tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://127.0.0.1:39509' processes=2 threads=4, memory=7.77 GiB>>
Traceback (most recent call last):
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/compatibility.py", line 161, in _run
    val = self.callback()
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/client.py", line 1442, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/home/evan.azevedo/.pyenv/versions/awsdx-hackathon/lib/python3.8/site-packages/distributed/batched.py", line 156, in send
    raise CommClosedError(f"Comm {self.comm!r} already closed.")
distributed.comm.core.CommClosedError: Comm <TCP (closed) Client->Scheduler local=tcp://127.0.0.1:57804 remote=tcp://127.0.0.1:39509> already closed.
2022-11-18 07:57:31,454 - tornado.application - ERROR - Exception in callback 

In [6]:
# Process the data in parallel using Dask
    (
        read_parquet([os.path.join(PARQUET_DIR, file) for file in files_todo])
        .map_partitions(
            tile_partition,
            meta={
                "geoid": pd.CategoricalDtype,
                "lat": np.float64,
                "lon": np.float64,
                "hex_fill": object,
            },
        )
        .to_parquet(os.path.join(DATA_DIR, "all_tiled_tracts.parquet"))
    )

2022-11-18 07:07:26,765 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-e8a0nuvr', purging
2022-11-18 07:07:45,004 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:33981 (pid=116672) exceeded 95% memory budget. Restarting...
2022-11-18 07:07:45,141 - distributed.nanny - WARNING - Restarting worker
2022-11-18 07:08:01,200 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:42613 (pid=116852) exceeded 95% memory budget. Restarting...
2022-11-18 07:08:01,347 - distributed.nanny - WARNING - Restarting worker
2022-11-18 07:08:19,700 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:43111 (pid=116892) exceeded 95% memory budget. Restarting...
2022-11-18 07:08:19,827 - distributed.nanny - WARNING - Restarting worker
2022-11-18 07:08:41,700 - distributed.worker_memory - WARNING - Worker tcp://127.0.0.1:46299 (pid=116942) exceeded 95% memory budget. Restarting...
2022-11-18 07:08:41,918 - distributed.nanny

KilledWorker: Attempted to run task ('to-parquet-09ec9af0e3eaf1addd9e01556a7caddc', 1) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:46299. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.